In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from dtaidistance import dtw
import itertools
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

import os

import random

from SVM_models.models import evaluate_binary_SVM_standard, evaluate_binary_SVM_custom

# import GunPoint dataset from GunPoint/GunPoint_test.txt and GunPoint/GunPoint_train.txt


def load_dataset(file_path):
    with open(file_path) as f:
        lines = f.readlines()
    # Parse each line, separating the label and features
    labels = []
    features = []
    for line in lines:
        values = list(map(float, line.strip().split()))
        labels.append(int(values[0])-1)  # First value is the label, cast to int
        features.append(values[1:])    # Remaining values are features
    # Convert lists to numpy arrays for easier handling
    labels = np.array(labels)
    features = np.array(features)
    # normalize dataset to 0 1
    features = (features - features.min()) / (features.max() - features.min())
    return labels, features

# Load the GunPoint dataset

#dataset_path with join

train_path = os.path.join("GunPoint", "GunPoint_TRAIN.txt")
test_path = os.path.join("GunPoint", "GunPoint_TEST.txt")

train_labels, train_features = load_dataset(train_path)
INITIAL_TEST_LABELS, INITIAL_TEST_FEATURES = load_dataset(test_path)

# split test in validation (50) and test (100)



In [2]:
EXPS = [
    "RANDOM",
    "RANDOM_BEST",
    "BAYESIAN_HAMILTONIAN",
    "BAYESIAN_MODEL",
]

OPT_TIMES = 30

CONFIG = {
    "N_QUBITS": 2,
    "SPARSITY": 0.95,
    "ENCODING": "euler",
}

SEEDS = range(10)
EXP = "RANDOM"


In [3]:
import os
def save_results(EXP, encoding, seed, train_accuracy, val_accuracy, test_accuracy, Hamiltonian_c, hidden_alphas):
    # Create folder structure
    folder_name = f"{EXP}__{encoding}"
    seed_folder = os.path.join("exps_results", folder_name, f"exp_{seed}")
    os.makedirs(seed_folder, exist_ok=True)

    # Save configuration and OPT_TIMES to a text file
    with open(os.path.join(seed_folder, "config.txt"), "w") as f:
        f.write(f"Experiment: {EXP}\n")
        f.write(f"Encoding: {encoding}\n")
        f.write(f"Number of Qubits: {CONFIG['N_QUBITS']}\n")
        f.write(f"Sparsity Coefficient: {CONFIG['SPARSITY']}\n")
        f.write(f"OPT_TIMES: {OPT_TIMES}\n")

    # Save train, validation, and test accuracy
    with open(os.path.join(seed_folder, "train_accuracy.txt"), "w") as f:
        f.write(f"Train Accuracy: {train_accuracy}\n")
    
    with open(os.path.join(seed_folder, "val_accuracy.txt"), "w") as f:
        f.write(f"Validation Accuracy: {val_accuracy}\n")

    with open(os.path.join(seed_folder, "test_accuracy.txt"), "w") as f:
        f.write(f"Test Accuracy: {test_accuracy}\n")

    # Save Hamiltonian_c and hidden_alphas
    with open(os.path.join(seed_folder, "Hamiltonian_c.txt"), "w") as f:
        f.write(str(Hamiltonian_c))

    with open(os.path.join(seed_folder, "hidden_alphas.txt"), "w") as f:
        f.write(str(hidden_alphas))


In [4]:
KERNELS = {"linear", "poly", "rbf", "sigmoid"}


In [5]:

for SEED in SEEDS:
    random.seed(SEED)
    test_labels, val_labels, test_features, val_features = train_test_split(INITIAL_TEST_LABELS, INITIAL_TEST_FEATURES, test_size=0.33, random_state=SEED)
    for kernel in KERNELS:
        EXP = kernel
        print(f"Evaluating SVM with kernel: {kernel}")
        results_dict = evaluate_binary_SVM_standard(train_features, val_features, test_features, train_labels, val_labels, test_labels, kernel_code=kernel, print_results=True)
        train_accuracy, val_accuracy, accuracy = results_dict["train_accuracy"], results_dict["validation_accuracy"], results_dict["test_accuracy"]

        save_results("Classical", EXP, SEED, train_accuracy, val_accuracy, accuracy, None, None)


Evaluating SVM with kernel: rbf
Train accuracy: 0.88
Validation accuracy: 0.76
Test accuracy: 0.76
Evaluating SVM with kernel: sigmoid
Train accuracy: 0.52
Validation accuracy: 0.44
Test accuracy: 0.52
Evaluating SVM with kernel: poly
Train accuracy: 1.0
Validation accuracy: 0.56
Test accuracy: 0.58
Evaluating SVM with kernel: linear
Train accuracy: 0.88
Validation accuracy: 0.82
Test accuracy: 0.71
Evaluating SVM with kernel: rbf
Train accuracy: 0.88
Validation accuracy: 0.74
Test accuracy: 0.77
Evaluating SVM with kernel: sigmoid
Train accuracy: 0.52
Validation accuracy: 0.48
Test accuracy: 0.5
Evaluating SVM with kernel: poly
Train accuracy: 1.0
Validation accuracy: 0.5
Test accuracy: 0.61
Evaluating SVM with kernel: linear
Train accuracy: 0.88
Validation accuracy: 0.72
Test accuracy: 0.76
Evaluating SVM with kernel: rbf
Train accuracy: 0.88
Validation accuracy: 0.84
Test accuracy: 0.72
Evaluating SVM with kernel: sigmoid
Train accuracy: 0.52
Validation accuracy: 0.48
Test accuracy: